In [28]:
#-------------------------------------IMPORTS-------------------------------------#
import torch
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA

In [27]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for word in title_word_list:
        word_features = extract_features(word)
        feature_matrix.append(word_features)
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0.] * NUM_FEATURES_PER_WORD] * (MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
# def cosine_similarity(question_one_batch_by_features, question_two_batch_by_features):
#     cosine_similarities = []
#     for idx in range(BATCH_SIZE):
#         vec_one = question_one_batch_by_features[idx]
#         vec_two = question_two_batch_by_features[idx]
#         numerator = np.dot(vec_one, vec_two)
#         denominator = LA.norm(vec_one)*LA.norm(vec_two)
#         value = numerator/denominator
#         cosine_similarities.append(value)
#     return cosine_similarities

# def cosine_similarity_worse(question_one_batch_by_features, question_two_batch_by_features):
#     print cosine_

In [4]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()

#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_list = convert_to_list(train_random_filename)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [8]:
import time
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:NUM_TRAINING_EXAMPLES])):   # we should include all but this is just for simplicity 
    if i%1000 == 0:
        print i

    sample = train_titles_only[i]
    
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
    
#     print "target_title: {}".format(target_title)
#     print "positive title: {}".format(positive_title)
#     print "negative titles: {}".format(negative_titles)
    
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
#     print "Target features shape: {}".format(target_features.shape)
#     print "Positive features shape: {}".format(positive_features.shape)
#     print "Negative features[0] shape: {}".format(n_negative_features[0].shape)
#     print "Num negative features: {}".format(len(n_negative_features)) 
    
    all_features = [target_features, positive_features] + n_negative_features
    features.append(all_features)

start = time.time()
print "Done Loop"
features = np.array(features) #speed up the following operations
targets = torch.LongTensor(len(features), 1).zero_()
training_dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
train_loader = data_utils.DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
print "Succesfully made the data batcher"

0
1000
2000
3000
4000
Done Loop
Succesfully made the data batcher


In [6]:
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 5000 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES

KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 600
LEARNING_RATE = 1e-3
NUM_EPOCHS = 25
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200
BATCH_SIZE = 50
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)

In [9]:
#-------------------------------------CNN-------------------------------------#
class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [43]:
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

net = CNN(INPUT_SIZE, HIDDEN_SIZE)
criterion = nn.MultiMarginLoss(p=1, margin=0.0, weight=None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

print "Beginning training..."
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        batch_cos_similarities = []
        batch_labels = []
        
        optimizer.zero_grad()
        
        target_question_features = sample[:, 0]
        positive_question_features = sample[:, 1]
        N_negative_question_features = sample[:, 2:]
        
        target_question_net_output = net(Variable(target_question_features))
        positive_question_net_output = net(Variable(positive_question_features))
        N_negative_question_net_outputs = [net(Variable(negative)) for negative in N_negative_question_features]
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative) for negative in N_negative_question_features]
                
        batch_cos_sims = []
        #50x600x38 --> 50x600x1. Averaging across the net outputs to get a single vector for each question.
        for sample_num in range(BATCH_SIZE):
            target_question_length = target_question_lengths[sample_num]
            positive_question_length = positive_question_lengths[sample_num]
            N_negative_question_lengths = N_negative_questions_lengths[sample_num]
            N_negative_question_outputs = N_negative_question_net_outputs[sample_num]
            
            target_question_vector = target_question_net_output.data.numpy()[sample_num,:,:target_question_length].mean(axis = 1)
            positive_question_vector = positive_question_net_output.data.numpy()[sample_num,:,:positive_question_length].mean(axis = 1)
            
            N_negative_question_vectors = [negative.data.numpy()[:,:N_negative_question_lengths[idx]].mean(axis = 1) for idx, negative in enumerate(N_negative_question_outputs)]
        

            cosine_similarity_pos = np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), positive_question_vector.reshape(1, -1)))
            cosine_similarity_neg = [np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), negative.reshape(1, -1))) for negative in N_negative_question_vectors]
            
            cosine_sims = np.array([cosine_similarity_pos] + cosine_similarity_neg).flatten()
            batch_cos_sims.append(cosine_sims)
        
        batch_labels = Variable(torch.from_numpy(label.numpy().squeeze()), requires_grad = True)
        loss = criterion(Variable(torch.FloatTensor(np.array(batch_cos_sims))), batch_labels)
            
        loss.backward()
            
        running_loss += loss.data[0]
            
        optimizer.step()

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
    
        
#         for batch_iter_idx in range(0, BATCH_SIZE):
#             current_sample_in_batch = sample[batch_iter_idx]
#             current_label = label[batch_iter_idx]

#             target_word_features = current_sample_in_batch[0]
#             positive_word_features = current_sample_in_batch[1]
#             N_negative_word_features = current_sample_in_batch[2:]
            
#             #!!!!!!
#             #FIX SHAPE OF FEATURES BECAUSE TORCH IS FUCKING WEIRD AND ONLY WANTS 3D INPUTS TO THE 1D CONV
#             #!!!!!!
#             target_word_features = torch.unsqueeze(target_word_features, 0)
#             positive_word_features = torch.unsqueeze(positive_word_features, 0)
#             N_negative_word_features = [torch.unsqueeze(negative_word_feature, 0) for negative_word_feature in N_negative_word_features]
#             #FINISHED FIXING SHAPE. FUCK TORCH
            
#             target_net_output = net(Variable(target_word_features, requires_grad = True))
#             positive_net_output = net(Variable(positive_word_features, requires_grad = True))
#             N_negative_word_features_net_output = [net(Variable(negative_word_feature, requires_grad = True)) for negative_word_feature in N_negative_word_features]
            

#             length_target_question = find_start_of_padding_single_vec(target_word_features.numpy()[0])
#             target_net_output_array = target_net_output.data.numpy().reshape((1, 21600)) / length_target_question
            
#             length_positive_question = find_start_of_padding_single_vec(positive_word_features.numpy()[0])
#             positive_net_output_array = positive_net_output.data.numpy().reshape((1, 21600)) / length_positive_question
            
#             length_negative_questions = [find_start_of_padding_single_vec(negative.numpy()[0]) for negative in N_negative_word_features]
#             N_negative_net_output_array = [negative.data.numpy().reshape((1, 21600)) / length_negative_questions[idx] for idx, negative in enumerate(N_negative_word_features_net_output)]
            
            
#             positive_cos_similarity = cosine_similarity(target_net_output_array, positive_net_output_array)
#             negative_cos_similarities = [cosine_similarity(target_net_output_array, negative) for negative in N_negative_net_output_array]
            
#             #WEIRD FORMATTING THINGS
#             positive_cos_similarity = positive_cos_similarity[0][0]
#             negative_cos_similarities = [negative[0][0] for negative in negative_cos_similarities]
            
#             cos_similarities = np.array([positive_cos_similarity] + negative_cos_similarities)
            
#             batch_cos_similarities.append(cos_similarities)
#             batch_labels.append(current_label.numpy())

        
#         #MORE WEIRD SHIT WITH WHAT TORCH NEEDS            
#         batch_cos_similarities = np.array(batch_cos_similarities)
#         batch_labels = np.array(batch_labels).reshape((BATCH_SIZE))
#         batch_cos_similarities = Variable(torch.from_numpy(batch_cos_similarities), requires_grad = True) 
#         batch_labels = Variable(torch.from_numpy(batch_labels), requires_grad = True)

#         loss = criterion(batch_cos_similarities, batch_labels)
            
#         loss.backward()
            
#         running_loss += loss.data[0]
            
#         optimizer.step()
            


Beginning training...
Loss after epoch 0 :0.800522348844
Loss after epoch 1 :0.800522353966
Loss after epoch 2 :0.800522349076
Loss after epoch 3 :0.800522387261


KeyboardInterrupt: 

In [15]:
positive_question_net_output.data.numpy().shape

(50, 600, 38)